In [1]:
from keras.applications import MobileNet

img_rows, img_cols = 224, 224


MobileNet = MobileNet(weights = 'imagenet',
                      include_top = False,
                      input_shape  =  (img_rows,  img_cols, 3))

## lets freeze the last  4 layers as they  are set to be trainable by  default
for layer in  MobileNet.layers:
  layer.trainable = False

## lets look at our layers
for (i, layer) in enumerate(MobileNet.layers):
  print(str(i)  + " "  +  layer.__class__.__name__, layer.trainable)

17227776/17225924 [==============================] - 2s 0us/step
0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False


In [2]:
## Let us  create a function to build our top layer

def addTopMobileNetLayer(bottom_model, num_classes):

  top_model = bottom_model.output
  top_model = GlobalAveragePooling2D()(top_model)
  top_model = (Dense(1024, activation = 'relu'))(top_model)
  top_model = (Dense(1024, activation = 'relu'))(top_model)
  top_model = (Dense(512, activation = 'relu'))(top_model)
  top_model = (Dense(num_classes, activation  = 'softmax'))(top_model)

  return top_model


In [3]:
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation,  Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 10

FC_head = addTopMobileNetLayer(MobileNet, num_classes)

model  = Model(inputs =  MobileNet.input, outputs = FC_head)

print(model.summary())


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

In [4]:
## Loading our  dataset

from keras.preprocessing.image import ImageDataGenerator

train_data_dir  =  'monkey_breed/train/'
validation_data_dir =  'monkey_breed/validation/'

## Lets perform data augmentation
train_datagen  = ImageDataGenerator(
    rescale =1./255,
    rotation_range = 45,
    width_shift_range =  0.3,
    height_shift_range =  0.3,
    horizontal_flip =  True,
    fill_mode =  'nearest'
)


validation_datagen  =  ImageDataGenerator(rescale  = 1./255)

## set our batch  size (16-32)
batch_size = 32

train_generator  = train_datagen.flow_from_directory(
    train_data_dir,
    target_size  = (img_rows,  img_cols),
    batch_size = batch_size,
    class_mode =  'categorical'
)

validation_generator  = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size  = (img_rows,  img_cols),
    batch_size = batch_size,
    class_mode =  'categorical'
)


Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [6]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("monkey_breed_mobileNet.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 1097
nb_validation_samples = 272

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
35/68 [==============>...............] - ETA: 35s - loss: 2.0753 - accuracy: 0.6667WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 340 batches). You may need to use the repeat() function when building your dataset.

Epoch 00001: val_loss improved from inf to 0.50179, saving model to monkey_breed_mobileNet.h5
35/68 [==============>...............] - 47s 1s/step - loss: 2.0753 - accuracy: 0.6667 - val_loss: 0.5018 - val_accuracy: 0.8382


In [24]:
from keras.models import load_model
import cv2
import os
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = ["mantled_howler", "patas_monkey","bald_uakari",
                     "japanese_macaque",
                      "pygmy_marmoset ",
                      "white_headed_capuchin",
                      "silvery_marmoset",
                      "common_squirrel_monkey",
                      "black_headed_night_monkey",
                      "nilgiri_langur"]

monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                      "n1": "patas_monkey",
                      "n2": "bald_uakari",
                      "n3": "japanese_macaque",
                      "n4": "pygmy_marmoset ",
                      "n5": "white_headed_capuchin",
                      "n6": "silvery_marmoset",
                      "n7": "common_squirrel_monkey",
                      "n8": "black_headed_night_monkey",
                      "n9": "nilgiri_langur"}

classifier = load_model('monkey_breed_mobileNet.h5')


def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("\nClass - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)


for i in range(0,10):
    input_im = getRandomImage("monkey_breed/validation/")
    
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    print("Prediction :  " , monkey_breeds_dict[res[0]])
    



Class - bald_uakari
Prediction :   mantled_howler

Class - silvery_marmoset
Prediction :   silvery_marmoset

Class - bald_uakari
Prediction :   bald_uakari

Class - nilgiri_langur
Prediction :   mantled_howler

Class - black_headed_night_monkey
Prediction :   black_headed_night_monkey

Class - japanese_macaque
Prediction :   japanese_macaque

Class - white_headed_capuchin
Prediction :   white_headed_capuchin

Class - silvery_marmoset
Prediction :   silvery_marmoset

Class - bald_uakari
Prediction :   bald_uakari

Class - mantled_howler 
Prediction :   mantled_howler
